In [9]:
# A command line interface(CLI) for searching the calories.csv file
import pandas as pd
from rapidfuzz import fuzz, process

# read the calorie data drom the CSV file
df = pd.read_csv('calories.csv') 

# function to perform fuzzy search on the 'FoodItem' column
def fuzzy_find_calories(df, search_term, n=1):
    matches = process.extract(search_term, df['FoodItem'], scorer=fuzz.token_set_ratio, limit=n)
    first_match = matches[0] # get the first(best) match
    food_item = first_match[0] # the food item from the first match
    match_score = first_match[1] # the similarity score of the first match
    index = first_match[2] # the index of the matching row in the dataframe

    print(f'searched for {search_term}\nbest match:{food_item}, {index}, {match_score}')

    return food_item, index, match_score

match_percent = 50 # set the default match percentage threshold

while True:
    # 1) ask the user for the food search term 
    search_term = input('Enter food item to search for: ')
    

    # 2) ask for a quality threshold (our match %), give it a default of 50. If the user just hits return, keep the existing value otherwise update it
    new_match_percent_str = input(f'Enter quality threshold (current: {match_percent}): ')
    if new_match_percent_str != '':
        match_percent = float(new_match_percent_str)

    # 3) search the FoodItem column in the DataFrame df for the search term with the fuzzy search
    food_item, index, match = fuzzy_find_calories(df, search_term)

    # 4) If match score exceeds the match threshold, print the food and calorie information
    if match > match_percent:
        row = df.iloc[index]
        # 4).a ask the user for the weight or portion size of the food
        weight_str = input(f'Enter the weight in grams of {row['FoodItem']}: ')
        weight = float(weight_str) if weight_str else 100 # default to 100 grams if no input

        # 4).b calculate total calories based on the portion size
        total_calories = (float(row['Kcals_per100grams']) * weight)/100

        print(f'The food {row['FoodItem']} has {total_calories} kcal for {weight} grams.')

    # 5) If match score is below the threshold, notify the user
    else:
        print(f'match of {match} is lower than {match_percent}, no good match found!')

    # 6) Finally ask the user if they want to search again. If they do, repeat the process. If not, exit the loop.
    new_search = input('Search again?(y/n): ')
    if new_search.lower() != 'y':
        break



searched for apple
best match:Apple, 30, 80.0
The food Apple has 26.0 kcal for 50.0 grams.
searched for banana
best match:Banana, 34, 83.33333333333333
The food Banana has 89.0 kcal for 100 grams.
